## Entendimiento de los datos

Vamos a trabajar con el conjunto de datos Churn-Modeling descargado de ***Kaggle***. Este conjunto de datos contiene detalles de los clientes de un banco y la variable objetivo es una variable binaria que refleja el hecho de si el cliente dejó el banco (cerró su cuenta) o si continúa siendo un cliente.

Las características capturan información sociodemográfica del cliente, información de productos financieros, el comportamiento y balance de su cuenta. Las características son las siguientes:

* **CustomerId**: Id único para identificar el cliente.
* **Surname**: Apellido del cliente.
* **CreditScore**: Puntaje de crédito del cliente.
* **Geography**: País al que pertenece el cliente.
* **Gender**: Genero.
* **Age**: Edad.
* **Tenure**: Número de años que el cliente ha estado en el banco.
* **Balance**: Saldo bancario del cliente.
* **NumOfProducts**: Número de productos bancarios que utiliza el cliente.
* **HasCrCard**: Si el cliente tiene tarjeta de crédito con el banco.
* **IsActiveMember**: Si el cliente es miembro activo del banco o no.
* **EstimatedSalary**: Salario estimado en dólares.
* **Exited**: 1-Si el cliente cerró la cuenta con el banco; 0-Si el cliente es retenido.


### Carga de módulos

In [1]:
# !pip install missingno
# Python library that provides the ability to understand the distribution of missing values through visualizations
# (Heat maps / Bar charts)

In [2]:
import warnings
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

warnings.filterwarnings(action="ignore")

In [3]:
churn = pd.read_csv("https://raw.githubusercontent.com/stivenlopezg/DS-ONLINE-76/master/data/churn-modeling.csv",dtype={"CustomerId": "category"})
churn.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,Yes,Yes,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,No,Yes,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,Yes,No,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,No,No,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,Yes,Yes,79084.10,0


In [4]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CustomerId       10000 non-null  category
 1   Surname          10000 non-null  object  
 2   CreditScore      10000 non-null  int64   
 3   Geography        9980 non-null   object  
 4   Gender           10000 non-null  object  
 5   Age              10000 non-null  int64   
 6   Tenure           10000 non-null  int64   
 7   Balance          10000 non-null  float64 
 8   NumOfProducts    10000 non-null  int64   
 9   HasCrCard        10000 non-null  object  
 10  IsActiveMember   10000 non-null  object  
 11  EstimatedSalary  9988 non-null   float64 
 12  Exited           10000 non-null  int64   
dtypes: category(1), float64(2), int64(5), object(5)
memory usage: 1.3+ MB


In [5]:
# Estadística descriptiva

churn.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9988.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,100066.908601,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,57519.993379,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,50910.677500,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,100185.240000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,199992.480000,1.000000


In [6]:
churn.describe(exclude="number")

,CustomerId,Surname,Geography,Gender,HasCrCard,IsActiveMember
count,10000,10000,9980,10000,10000,10000
unique,10000,2932,3,2,2,2
top,15565701,Smith,France,Male,Yes,Yes
freq,1,32,5008,5457,7055,5151


#Datos missing
Vamos a mirar si hay datos missing en nuestro set de datos

In [7]:
# Conteo
churn.isna().sum()

CustomerId          0
Surname             0
CreditScore         0
Geography          20
Gender              0
Age                 0
Tenure              0
Balance             0
NumOfProducts       0
HasCrCard           0
IsActiveMember      0
EstimatedSalary    12
Exited              0
dtype: int64

In [8]:
# Lo mismo, en porcentaje
churn.isna().mean() * 100

CustomerId         0.00
Surname            0.00
CreditScore        0.00
Geography          0.20
Gender             0.00
Age                0.00
Tenure             0.00
Balance            0.00
NumOfProducts      0.00
HasCrCard          0.00
IsActiveMember     0.00
EstimatedSalary    0.12
Exited             0.00
dtype: float64

### Preprocesamiento

* Variables numéricas:
    * Atípicos
    * Imputar
    * Escalar
    * Discretizar (Opcional)

* Variables categóricas:
    * Imputar
    * Codificar (OHE, LabelEncoder, u OrdinalEncoder)

In [9]:
#Decidimos que estas columnas confunden al modelo o no tienen correlacion con la var que queremos predecir
cols_to_drop = ["CustomerId", "Surname"]

# axis = 0 se refiere a rows, axis=1 a columns
# Para major claridad, se puede usar axis='index' para rows y axis='columns'

churn.drop(labels=cols_to_drop, axis='columns', inplace=True)

churn.sample(n=1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
4489,602,France,Female,35,8,0.0,2,Yes,Yes,152843.53,0


In [10]:
#Lista de features de tipo numérico
numerical_features = churn.select_dtypes(include="number").columns.tolist()

# Removemos la variable target, que queremos predecir
numerical_features.remove("Exited")
numerical_features

['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [11]:
#Lista de features catégoricos, a partir de excluír los que no son number
categorical_features = churn.select_dtypes(exclude="number").columns.tolist()
categorical_features

['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']

In [12]:
#Pop: method to delete any element from the dataframe
exited = churn.pop("Exited")

# train_data, test_data, train_label, test_label

#test_size refers to the data percentage used for testing. If 0.3, 0.7 is the one used for training.
X_train, X_test, y_train, y_test = train_test_split(churn, exited,
                                                    test_size=0.3, random_state=42)

In [13]:
X_train.isna().sum()

CreditScore         0
Geography           0
Gender              0
Age                 0
Tenure              0
Balance             0
NumOfProducts       0
HasCrCard           0
IsActiveMember      0
EstimatedSalary    10
dtype: int64

#### Preprocesamiento sobre los features numéricos

In [14]:
#Imputacion - Rellenar los datos faltantes - estrategias:
 # mean: replace missing values using the mean along each column
 # median: replace missing values using the median along each column
 # most_frequent: replace missing using the most frequent value along each column.
    # Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.
 # constant: replace missing values with fill_value.

# selecciona de X_train solo aquellas columnas que están contenidas en la lista "numerical_features list
imputer_num = SimpleImputer(strategy="median").fit(X_train[numerical_features])

#Valor de la mediana con la que imputó cada feature.
for feature, median in zip(numerical_features, imputer_num.statistics_):
    print(f"La mediana para {feature} es: {round(median, 2)}")

La mediana para CreditScore es: 653.0
La mediana para Age es: 37.0
La mediana para Tenure es: 5.0
La mediana para Balance es: 96483.19
La mediana para NumOfProducts es: 1.0
La mediana para EstimatedSalary es: 100853.09


In [15]:
# Transform: aplica sobre las columnas numericas los valores resultado de la
# imputación anterior - en este caso la media - sobre los valores faltantes
X_train.loc[:, numerical_features] = imputer_num.transform(X_train[numerical_features])

#Vemos que EstimatedSalary ya no tiene nulos. Resta trabajar sobre Geography, que es un valor categórico
X_train.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [16]:
# Escalar: util cuando sigue una distribución normal.
# StandardScaler estandariza un feature al restarle el promedio y luego dividir
# por la desviacion estandar, obteniendo "unidades" de desviacion estandar.
# Así, el promedio de la distribucion es 0 y alrededor del 68% de los valores
# estarán entre -1 and 1.

scaler = StandardScaler().fit(X_train[numerical_features])

X_train.loc[:, numerical_features] = scaler.transform(X_train[numerical_features])

In [17]:
# Podemos ver que se aplicó a todas las columnas numericas y que ya los valores
# ya no tienen la misma interpretacion que al momento de importar el dataset
X_train.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
9069,-0.344595,Spain,Female,-0.656750,-0.342170,1.583725,0.819663,Yes,Yes,1.248986
2603,-0.095181,Germany,Female,-0.466380,0.698162,1.344106,-0.903352,No,Yes,1.522114
7738,-0.947345,France,Male,-0.561565,0.351385,-1.222055,0.819663,No,No,1.264394
1579,-0.354987,Germany,Male,0.199916,1.044940,-0.618965,-0.903352,Yes,Yes,1.647781
5058,0.642668,France,Male,-0.180824,1.391718,1.152808,0.819663,No,Yes,0.875726


#### Preprocesamiento Variables categóricas

In [18]:
# Imputacion

# Sobre datos categóricos, la estrategia puede ser la "más frecuente" o el uso de una "constante"

imputer_cat = SimpleImputer(strategy="most_frequent").fit(X_train[categorical_features])

# Para cada variable categórica, arma un array con el valor más frecuente
for feature, most_freq in zip(categorical_features, imputer_cat.statistics_):
    print(f"Para la variable {feature}, el valor mas frecuente es: {most_freq}")

X_train.loc[:, categorical_features] = imputer_cat.transform(X_train[categorical_features])

#Geography pasó a tener cero valores faltantes. Segun vimos antes, el top value era Francia
X_train.isna().sum()

Para la variable Geography, el valor mas frecuente es: France
Para la variable Gender, el valor mas frecuente es: Male
Para la variable HasCrCard, el valor mas frecuente es: Yes
Para la variable IsActiveMember, el valor mas frecuente es: Yes


CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [19]:
# One Hot Encoder
#Para variables nominales sin jerarquía, crea una columna por cada uno de los valores encontrado
#asignando una variable dummy - que puede tomar dos posibles valores: 1 o 0 - para indicar
#la presencia o ausencia de esa categoria en esa observación

## no le está gustando la lista categorical_features como param de columns,
#me tira error: raise TypeError("Input must be a list-like for parameter `columns`")
#X_train = pd.get_dummies(data=X_train, columns=categorical_features)

#Se lo saqué. La doc dice que por default, toda columna de dtype object o categorical se lo aplica
#si el parámetro no se especifica.
X_train = pd.get_dummies(data=X_train)
X_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,HasCrCard_No,HasCrCard_Yes,IsActiveMember_No,IsActiveMember_Yes
9069,-0.344595,-0.656750,-0.342170,1.583725,0.819663,1.248986,0,0,1,1,0,0,1,0,1
2603,-0.095181,-0.466380,0.698162,1.344106,-0.903352,1.522114,0,1,0,1,0,1,0,0,1
7738,-0.947345,-0.561565,0.351385,-1.222055,0.819663,1.264394,1,0,0,0,1,1,0,1,0
1579,-0.354987,0.199916,1.044940,-0.618965,-0.903352,1.647781,0,1,0,0,1,0,1,0,1
5058,0.642668,-0.180824,1.391718,1.152808,0.819663,0.875726,1,0,0,0,1,1,0,0,1


### Set de Test
#### Preprocesamiento Variables Numéricas

In [20]:
#Conteo de nulos por columna
X_test.isna().sum()

CreditScore         0
Geography          20
Gender              0
Age                 0
Tenure              0
Balance             0
NumOfProducts       0
HasCrCard           0
IsActiveMember      0
EstimatedSalary     2
dtype: int64

##### Imputación

In [21]:
# imputer_num: Definida durante el training, array con la imputación a aplicar sobre numericos
X_test.loc[:, numerical_features] = imputer_num.transform(X_test[numerical_features])

#Columnas originalmente con valores faltantes, ya no tienen más
X_test.isna().sum()

CreditScore         0
Geography          20
Gender              0
Age                 0
Tenure              0
Balance             0
NumOfProducts       0
HasCrCard           0
IsActiveMember      0
EstimatedSalary     0
dtype: int64

##### Escalado sobre el set de Test

In [22]:
#Scaler

X_test.loc[:, numerical_features] = scaler.transform(X_test[numerical_features])

#No veo que aplique el scaler a los features numericos.
X_test.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
6252,-0.583617,NaN,Male,-0.656750,-0.688948,0.324894,0.819663,No,No,-1.024156
4684,-0.303026,NaN,Male,0.390286,-1.382503,-1.222055,0.819663,Yes,Yes,0.790674
1731,-0.531655,NaN,Female,0.485471,-0.342170,-1.222055,0.819663,Yes,No,-0.733117
4742,-1.518919,NaN,Male,1.913248,1.044940,0.683891,0.819663,Yes,Yes,1.212328
4521,-0.957737,NaN,Female,-1.132675,0.698162,0.777369,-0.903352,Yes,Yes,0.240460


### Set de Test
#### Preprocesamiento Variables categóricas

In [23]:
X_test.loc[:, categorical_features] = imputer_cat.transform(X_test[categorical_features])

#Geography pasó a tener cero valores faltantes. Segun vimos antes, el top value era Francia
X_test.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

#### One Hot Encoder

In [24]:
X_test = pd.get_dummies(data=X_test)
X_test.sample(4)


,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,HasCrCard_No,HasCrCard_Yes,IsActiveMember_No,IsActiveMember_Yes
8674,-1.279897,2.674729,1.044940,0.427148,0.819663,0.922941,1,0,0,0,1,0,1,0,1
1614,-0.115966,3.626580,1.391718,-1.222055,2.542677,-0.334848,1,0,0,0,1,0,1,0,1
5454,0.538746,0.009546,0.004607,-1.222055,-0.903352,-1.637911,1,0,0,0,1,0,1,1,0
5709,-0.084789,0.009546,1.391718,-1.222055,-0.903352,-1.684344,1,0,0,1,0,0,1,1,0
